In [206]:
import json
from enum import Enum

# Define EntityType Enum
class EntityType(Enum):
    BOUNDARY = "boundary"
    PLAYER = "player"
    OBJECT = "object"
    REGION = "region"

# Define ArenaEntity class
class ArenaEntity:
    def __init__(self, color, shape, tag, aruco_id, entity_id, entity_type, mobility):
        self.color = color
        self.shape = shape
        self.tag = tag
        self.aruco_id = aruco_id
        self.id = entity_id
        self.entity_type = entity_type  # Entity type using Enum (BOUNDARY, PLAYER, OBJECT, REGION)
        self.mobility = mobility  # Parameter indicating movement capability

    def __repr__(self):
        return (f"ArenaEntity(id={self.id}, aruco_id={self.aruco_id}, entity_type={self.entity_type.name}, "
                f"color={self.color}, shape={self.shape}, tag={self.tag}, mobility={self.mobility})")

# Function to parse the JSON configuration and return a list of ArenaEntity objects
def create_entities_from_config(json_config):
    entities = []
    entity_id_counter = 1  # Start the entity IDs from 1

    # Mapping of entity_type names to the EntityType enum
    entity_type_mapping = {
        "boundary": EntityType.BOUNDARY,
        "player": EntityType.PLAYER,
        "object": EntityType.OBJECT,
        "region": EntityType.REGION
    }

    # Loop through each entity_type in the JSON config
    for entity_type_key, objects in json_config.items():
        entity_type = entity_type_mapping.get(entity_type_key)  # Convert to EntityType Enum

        # Iterate through the list of objects for the current entity_type
        for obj in objects:
            count = obj.get("count", 1)  # Default count to 1 if not provided
            for _ in range(count):
                # Create an ArenaEntity object for each entity and give it a unique id
                entity = ArenaEntity(
                    entity_id = obj.get("id"),
                    color=obj.get("color"),
                    shape=obj.get("shape"),
                    tag=obj.get("tags"),
                    aruco_id=obj.get("aruco_id"),  # Assuming aruco_id isn't provided in the JSON, can add logic if needed
                    entity_type=entity_type,
                    mobility=obj.get("mobility")
                )
                entities.append(entity)
                entity_id_counter += 1  # Increment the entity ID for each entity

    return entities

# Function to read JSON from file
def read_json_from_file(file_path):
    with open(file_path, "r") as file:
        json_data = json.load(file)
    return json_data

# Example usage:
json_file_path = "config/config3.json"  # Specify the path to your JSON file
json_config = read_json_from_file(json_file_path)

# Create entities from the config
created_entities = create_entities_from_config(json_config)

# Display the created entities
for entity in created_entities:
    print(entity)


ArenaEntity(id=1, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary1, mobility=None)
ArenaEntity(id=2, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary2, mobility=None)
ArenaEntity(id=3, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary3, mobility=None)
ArenaEntity(id=4, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary4, mobility=None)
ArenaEntity(id=5, aruco_id=None, entity_type=PLAYER, color=green, shape=triangle, tag=playerA, mobility=None)
ArenaEntity(id=6, aruco_id=None, entity_type=PLAYER, color=red, shape=triangle, tag=playerB, mobility=None)


In [207]:
import numpy as np
import cv2
from enum import Enum
from shapely.geometry import MultiPoint, Polygon

# Define color ranges (HSV format)
COLOR_RANGES = {
    "red": [
        (np.array([0, 50, 50]), np.array([10, 255, 255])),  # Lower red range
        (np.array([170, 50, 50]), np.array([180, 255, 255]))  # Upper red range
    ],
    "blue": [
        (np.array([100, 50, 50]), np.array([130, 255, 255])),
    ],
    "green": [
        (np.array([40, 50, 50]), np.array([80, 255, 255]))
    ],
    "yellow": [
        (np.array([20, 100, 100]), np.array([30, 255, 255]))
    ],
    "cyan": [
        (np.array([80, 100, 100]), np.array([90, 255, 255]))
    ],
    "magenta": [
        (np.array([140, 50, 50]), np.array([160, 255, 255]))
    ],
    "orange": [
        (np.array([10, 100, 100]), np.array([20, 255, 255]))
    ],
    "purple": [
        (np.array([125, 50, 50]), np.array([140, 255, 255]))
    ],
    "black": [
        (np.array([0, 0, 0]), np.array([180, 255, 30]))
    ],
    "white": [
        (np.array([0, 0, 200]), np.array([180, 30, 255]))
    ]
}


In [209]:
 # Compute the concave hull
def compute_concave_hull(points, alpha=0.1):
    if len(points) < 4:
        return points  # Not enough points for a hull
    points = MultiPoint(points)
    polygon = points.convex_hull
    return polygon


# Working

In [27]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

height = 500
width = 800

while True:
    ret, frame = cam.read()
    if not ret:
        break

    cv2.imshow("Frame",frame)



    if cv2.waitKey(1) == ord("q"):
        break

cam.release()
cv2.destroyAllWindows()

In [210]:
def detect_shapes_and_colors(image):

    detected_shapes = []
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    for color_name, ranges in COLOR_RANGES.items():
        # Create a combined mask for the color
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        for lower, upper in ranges:
            mask = cv2.bitwise_or(mask, cv2.inRange(hsv_image, lower, upper))

        # Find contours for the color
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            if cv2.contourArea(contour) > 100:  # Filter small contours
                # Approximate the contour to detect shapes
                epsilon = 0.02 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)
                shape = "unknown"

                # Identify shape based on the number of vertices
                num_vertices = len(approx)
                if num_vertices > 7:
                    shape = "circle"
                elif num_vertices == 3:
                    shape = "triangle"
                elif num_vertices == 4:
                    # Further check for rectangle or square
                    (x, y, w, h) = cv2.boundingRect(approx)
                    aspect_ratio = float(w) / h
                    if 0.9 <= aspect_ratio <= 1.1:
                        shape = "square"
                    else:
                        shape = "rectangle"
                elif num_vertices == 2:
                    shape = "line"

                # Get points for the detected shape
                points = [tuple(point[0]) for point in approx]
                detected_shapes.append(((shape, color_name), points))

    return detected_shapes

In [221]:
def detect_aruco_markers(image):

    markerDictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
    detectorParam = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(markerDictionary, detectorParam)
    corners, ids, rejected = detector.detectMarkers(image)

    list_of_aruco = []
    if ids is not None :
        for i in range(len(ids)):
            list_of_aruco.append((ids[i][0],corners[i][0]))

    return list_of_aruco

In [222]:
image = cv2.imread('test_images/shapes3.png')  #
shape_color_points = detect_shapes_and_colors(image)
aruco_marker_points = detect_aruco_markers(image)
print([x[0] for x in shape_color_points])
print([x[0] for x in aruco_marker_points])

[('circle', 'red'), ('circle', 'red'), ('triangle', 'red'), ('circle', 'red'), ('circle', 'red'), ('triangle', 'green'), ('rectangle', 'white')]
[]


In [223]:
from collections import defaultdict

def assign_positions_to_color_entities(entities, shape_color_points):

    # Grouping entities by shape and color
    grouped_entities = defaultdict(list)
    for entity in entities:
        grouped_entities[(entity.shape, entity.color)].append(entity)

    # Dictionary to store available positions by shape and color
    positions_dict = defaultdict(list)
    for (shape, color), points in shape_color_points:
        positions_dict[(shape, color)].append(points)
    # print([len(x) for x in positions_dict.values()])

    # Prepare result list
    result = []

    # For each shape/color group, assign points to each entity in the group
    for shape_color, entities_group in grouped_entities.items():
        shape, color = shape_color
        
        if len(grouped_entities[(shape, color)]) <= len(positions_dict[(shape, color)]):
            for i in range(len(grouped_entities[(shape, color)])):
                positions = positions_dict[(shape, color)].pop()
                entity = grouped_entities[(shape, color)].pop()
                result.append((entity,positions))
            
    return result

In [224]:
def assign_positions_to_aruco_entities(entities, list_of_aruco):

    # Grouping entities by shape and color
    grouped_entities = defaultdict(list)
    for entity in entities:
        grouped_entities[entity.aruco_id].append(entity)

    # Dictionary to store available positions by shape and color
    positions_dict = defaultdict(list)
    for id, points in list_of_aruco:
        positions_dict[id].append(points)
    # print([len(x) for x in positions_dict.values()])

    # Prepare result list
    result = []

    # For each shape/color group, assign points to each entity in the group
    for id, entities_group in grouped_entities.items():
        if len(grouped_entities[id]) <= len(positions_dict[id]):
            for i in range(len(grouped_entities[id])):
                positions = positions_dict[id].pop()
                entity = grouped_entities[id].pop()
                result.append((entity,positions))
            
    return result

In [225]:
aruco_entities = []
color_entities = []

for entity in created_entities:
    if entity.aruco_id!=None:
        aruco_entities.append(entity)
    if entity.color != None:
        color_entities.append(entity)

print(aruco_entities)
print(color_entities)


[]
[ArenaEntity(id=1, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary1, mobility=None), ArenaEntity(id=2, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary2, mobility=None), ArenaEntity(id=3, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary3, mobility=None), ArenaEntity(id=4, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary4, mobility=None), ArenaEntity(id=5, aruco_id=None, entity_type=PLAYER, color=green, shape=triangle, tag=playerA, mobility=None), ArenaEntity(id=6, aruco_id=None, entity_type=PLAYER, color=red, shape=triangle, tag=playerB, mobility=None)]


In [226]:
color_entity_pos = assign_positions_to_color_entities(created_entities, shape_color_points)
aruco_entity_pos = assign_positions_to_aruco_entities(aruco_entities,aruco_marker_points)
# pprint.pp([x[0] for x in entity_pos])


entity_pos = []
entity_pos.extend(color_entity_pos)
entity_pos.extend(aruco_entity_pos)

print(entity_pos)

[(ArenaEntity(id=4, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary4, mobility=None), [(731, 102), (714, 126), (728, 157), (759, 165), (788, 149), (794, 126), (780, 104), (759, 96)]), (ArenaEntity(id=3, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary3, mobility=None), [(235, 103), (216, 121), (217, 140), (232, 154), (256, 154), (271, 140), (271, 118), (253, 103)]), (ArenaEntity(id=2, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary2, mobility=None), [(602, 340), (583, 351), (577, 374), (588, 393), (611, 399), (629, 388), (635, 365), (624, 346)]), (ArenaEntity(id=1, aruco_id=None, entity_type=BOUNDARY, color=red, shape=circle, tag=boundary1, mobility=None), [(203, 362), (185, 374), (180, 397), (193, 418), (211, 422), (229, 410), (234, 387), (221, 366)]), (ArenaEntity(id=5, aruco_id=None, entity_type=PLAYER, color=green, shape=triangle, tag=playerA, mobility=None), [(557, 135), (533, 181), (582, 182)]), (Aren

In [227]:
def process_boundary_data(list_of_entity_pos):

    all_points = []

    for entity, points in list_of_entity_pos:
        # Calculate the center point of the boundary entity
        center = np.mean(points, axis=0)
        all_points.append(center)

    if len(all_points) == 0:
        print("No center points found for boundary entities.")
        return None

    # Compute convex hull of the center points
    all_points = np.array(all_points)
    convex_hull = compute_concave_hull(all_points)
    hull_points = np.array(list(convex_hull.exterior.coords), dtype=np.int32)

    return hull_points[:-1]

In [257]:
def categorize_entity(entity_pos):

    # Initialize categorized entity lists
    entity_categories = {
        EntityType.BOUNDARY: [],
        EntityType.PLAYER: [],
        EntityType.OBJECT: [],
        EntityType.REGION: [],
    }

    # Categorize entities
    for (entity,pos) in entity_pos:
        if entity.entity_type in entity_categories:
            entity_categories[entity.entity_type].append((entity,np.array(pos)))
        else:
            print(f"Unknown entity type: {entity.entity_type}")

    boundary_points = process_boundary_data(entity_categories[EntityType.BOUNDARY])
    entity_categories[EntityType.BOUNDARY] = boundary_points

    return entity_categories

In [263]:
def transform_points(points, M):

    points_homogeneous = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed_points_homogeneous = np.dot(points_homogeneous, M.T)
    transformed_points = transformed_points_homogeneous[:, :2] / transformed_points_homogeneous[:, 2][:, np.newaxis]
    return np.array(transformed_points,dtype=np.int32)

In [264]:
def get_6dof_pos(points, is_aruco=False):

    if is_aruco:
        x = (points[0][0] + points[2][0]) / 2
        y = (points[0][1] + points[2][1]) / 2
        yaw = np.arctan2(points[2][1] - points[0][1], points[2][0] - points[0][0])
        return np.array((x, y, 0, 0, 0, yaw),dtype=np.int32)
    
    
    else:
        point = np.mean(points,axis=0)
        return np.array((point[0], point[1], 0, 0, 0, 0),dtype=np.int32)


In [265]:

def process_entity_data(entity_data, M):

    for entity_type in entity_data:
        if entity_type  == EntityType.BOUNDARY:
            tf_pos = transform_points(entity_data[EntityType.BOUNDARY],M)
            entity_data[EntityType.BOUNDARY] = tf_pos
        else:
            process_entity_pos = []
            entity_pos = entity_data[entity_type]
            for entity,pos in entity_pos:
                if len(pos) > 0:
                    tf_pos = transform_points(pos,M)
                    if entity_type in [EntityType.OBJECT,EntityType.PLAYER] :
                        tf_pose = get_6dof_pos(tf_pos,entity.aruco_id!=None)
                    process_entity_pos.append((entity,tf_pose))

                entity_data[entity_type] = process_entity_pos

    return entity_data

In [266]:
entity_data = categorize_entity(entity_pos)
entity_data

{<EntityType.BOUNDARY: 'boundary'>: array([[756, 128],
        [243, 129],
        [207, 392],
        [606, 369]]),
 <EntityType.PLAYER: 'player'>: [(ArenaEntity(id=5, aruco_id=None, entity_type=PLAYER, color=green, shape=triangle, tag=playerA, mobility=None),
   array([[557, 135],
          [533, 181],
          [582, 182]], dtype=int32)),
  (ArenaEntity(id=6, aruco_id=None, entity_type=PLAYER, color=red, shape=triangle, tag=playerB, mobility=None),
   array([[367, 205],
          [339, 256],
          [396, 257]], dtype=int32))],
 <EntityType.OBJECT: 'object'>: [],
 <EntityType.REGION: 'region'>: []}

In [267]:
def order_points(pts):

    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)

    rect[0] = pts[np.argmin(s)]  # top-left: smallest sum of x + y
    rect[2] = pts[np.argmax(s)]  # bottom-right: largest sum of x + y
    rect[1] = pts[np.argmin(diff)]  # top-right: smallest difference of x - y
    rect[3] = pts[np.argmax(diff)]  # bottom-left: largest difference of x - y

    return rect

In [268]:
height = 400
width = 800
dst_pts = order_points(np.float32([[0, 0], [width, 0], [width, height], [0, height]]))
src_pts = order_points(np.float32(entity_data[EntityType.BOUNDARY]))
# Calculate the perspective transform matrix
M = cv2.getPerspectiveTransform(src_pts, dst_pts)

In [269]:
process_entity_data(entity_data,M)

{<EntityType.BOUNDARY: 'boundary'>: array([[800,   0],
        [  0,   0],
        [  0, 400],
        [800, 400]]),
 <EntityType.PLAYER: 'player'>: [(ArenaEntity(id=5, aruco_id=None, entity_type=PLAYER, color=green, shape=triangle, tag=playerA, mobility=None),
   array([495,  48,   0,   0,   0,   0])),
  (ArenaEntity(id=6, aruco_id=None, entity_type=PLAYER, color=red, shape=triangle, tag=playerB, mobility=None),
   array([224, 146,   0,   0,   0,   0]))],
 <EntityType.OBJECT: 'object'>: [],
 <EntityType.REGION: 'region'>: []}

In [271]:
warped_img = cv2.warpPerspective(image, M, (width, height))
cv2.imshow("Transformed",warped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()